## Exercise solution example - Create a prototype notebook
Make a notebook where you:
- load the file with pandas
- check for NaNs
- figure out how you remove the NaNs
- double check how many rows you throw away. Find a solution if this doesnt look good.
- clean up the column with the names of the penguins. They are too long, so shorten them with a regular expression
- save the cleaned file with a timestamp

#### First import packages

In [1]:
import numpy as np
import pandas as pd
from pathlib import Path


#### Set path variables

In [2]:
datadir = Path("../data/raw/") # The 2 dots before /data means go one folder up
outputdir = Path("../data/processed/")
filename = datadir / "palmerpenguins.parq"
filename.resolve(), filename.exists()


(PosixPath('/home/admindme/code/cleanup/data/raw/palmerpenguins.parq'), True)

#### Read the parquet file and show same basic info, make sure you have the pyarrow module installed: poetry add pyarrow

In [3]:
# Load parquet file
df = pd.read_parquet(filename, engine="pyarrow")

# Show some basic info about the penguins file
df.describe()

,Sample Number,Culmen Length (mm),Culmen Depth (mm),Flipper Length (mm),Body Mass (g),Delta 15 N (o/oo),Delta 13 C (o/oo)
count,344.000000,342.000000,342.000000,342.000000,342.000000,330.000000,331.000000
mean,63.151163,43.921930,17.151170,200.915205,4201.754386,8.733382,-25.686292
std,40.430199,5.459584,1.974793,14.061714,801.954536,0.551770,0.793961
min,1.000000,32.100000,13.100000,172.000000,2700.000000,7.632200,-27.018540
25%,29.000000,39.225000,15.600000,190.000000,3550.000000,8.299890,-26.320305
50%,58.000000,44.450000,17.300000,197.000000,4050.000000,8.652405,-25.833520
75%,95.250000,48.500000,18.700000,213.000000,4750.000000,9.172123,-25.062050
max,152.000000,59.600000,21.500000,231.000000,6300.000000,10.025440,-23.787670


In [4]:
# with df.info() you can see all of the column names and what kind of
# object the column consists of: object, int64, float64 etc.
# take notice of the non-null count column. Some columns consists of
# 344 rows, others e.g. 342,333, 330 etc. - > meaning some of these rows
# have NaN values

df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 344 entries, 0 to 343
Data columns (total 17 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   studyName            344 non-null    object 
 1   Sample Number        344 non-null    int64  
 2   Species              344 non-null    object 
 3   Region               344 non-null    object 
 4   Island               344 non-null    object 
 5   Stage                344 non-null    object 
 6   Individual ID        344 non-null    object 
 7   Clutch Completion    344 non-null    object 
 8   Date Egg             344 non-null    object 
 9   Culmen Length (mm)   342 non-null    float64
 10  Culmen Depth (mm)    342 non-null    float64
 11  Flipper Length (mm)  342 non-null    float64
 12  Body Mass (g)        342 non-null    float64
 13  Sex                  333 non-null    object 
 14  Delta 15 N (o/oo)    330 non-null    float64
 15  Delta 13 C (o/oo)    331 non-null    flo

In [5]:
# df.head() shows the first five rows of the data table
df.head()

,studyName,Sample Number,Species,Region,Island,Stage,Individual ID,Clutch Completion,Date Egg,Culmen Length (mm),Culmen Depth (mm),Flipper Length (mm),Body Mass (g),Sex,Delta 15 N (o/oo),Delta 13 C (o/oo),Comments
0,PAL0708,1,Adelie Penguin (Pygoscelis adeliae),Anvers,Torgersen,"Adult, 1 Egg Stage",N1A1,Yes,2007-11-11,39.1,18.7,181.0,3750.0,MALE,NaN,NaN,Not enough blood for isotopes.
1,PAL0708,2,Adelie Penguin (Pygoscelis adeliae),Anvers,Torgersen,"Adult, 1 Egg Stage",N1A2,Yes,2007-11-11,39.5,17.4,186.0,3800.0,FEMALE,8.94956,-24.69454,None
2,PAL0708,3,Adelie Penguin (Pygoscelis adeliae),Anvers,Torgersen,"Adult, 1 Egg Stage",N2A1,Yes,2007-11-16,40.3,18.0,195.0,3250.0,FEMALE,8.36821,-25.33302,None
3,PAL0708,4,Adelie Penguin (Pygoscelis adeliae),Anvers,Torgersen,"Adult, 1 Egg Stage",N2A2,Yes,2007-11-16,NaN,NaN,NaN,NaN,None,NaN,NaN,Adult not sampled.
4,PAL0708,5,Adelie Penguin (Pygoscelis adeliae),Anvers,Torgersen,"Adult, 1 Egg Stage",N3A1,Yes,2007-11-16,36.7,19.3,193.0,3450.0,FEMALE,8.76651,-25.32426,None


#### First we check which of the colums consist of Nan Values

In [6]:
select = list(df.isna().sum() > 0)
select

[False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True]

#### With this above produced list, we delete the columns which contains of NaNs

In [7]:
df = df.dropna(subset=df.columns[select], axis="rows")
df


,studyName,Sample Number,Species,Region,Island,Stage,Individual ID,Clutch Completion,Date Egg,Culmen Length (mm),Culmen Depth (mm),Flipper Length (mm),Body Mass (g),Sex,Delta 15 N (o/oo),Delta 13 C (o/oo),Comments
6,PAL0708,7,Adelie Penguin (Pygoscelis adeliae),Anvers,Torgersen,"Adult, 1 Egg Stage",N4A1,No,2007-11-15,38.9,17.8,181.0,3625.0,FEMALE,9.18718,-25.21799,Nest never observed with full clutch.
7,PAL0708,8,Adelie Penguin (Pygoscelis adeliae),Anvers,Torgersen,"Adult, 1 Egg Stage",N4A2,No,2007-11-15,39.2,19.6,195.0,4675.0,MALE,9.46060,-24.89958,Nest never observed with full clutch.
28,PAL0708,29,Adelie Penguin (Pygoscelis adeliae),Anvers,Biscoe,"Adult, 1 Egg Stage",N18A1,No,2007-11-10,37.9,18.6,172.0,3150.0,FEMALE,8.38404,-25.19837,Nest never observed with full clutch.
29,PAL0708,30,Adelie Penguin (Pygoscelis adeliae),Anvers,Biscoe,"Adult, 1 Egg Stage",N18A2,No,2007-11-10,40.5,18.9,180.0,3950.0,MALE,8.90027,-25.11609,Nest never observed with full clutch.
38,PAL0708,39,Adelie Penguin (Pygoscelis adeliae),Anvers,Dream,"Adult, 1 Egg Stage",N25A1,No,2007-11-13,37.6,19.3,181.0,3300.0,FEMALE,9.41131,-25.04169,Nest never observed with full clutch.
68,PAL0809,69,Adelie Penguin (Pygoscelis adeliae),Anvers,Torgersen,"Adult, 1 Egg Stage",N32A1,No,2008-11-11,35.9,16.6,190.0,3050.0,FEMALE,8.47781,-26.07821,Nest never observed with full clutch.
69,PAL0809,70,Adelie Penguin (Pygoscelis adeliae),Anvers,Torgersen,"Adult, 1 Egg Stage",N32A2,No,2008-11-11,41.8,19.4,198.0,4450.0,MALE,8.86853,-26.06209,Nest never observed with full clutch.
120,PAL0910,121,Adelie Penguin (Pygoscelis adeliae),Anvers,Torgersen,"Adult, 1 Egg Stage",N66A1,No,2009-11-17,36.2,17.2,187.0,3150.0,FEMALE,9.04296,-26.19444,Nest never observed with full clutch.
121,PAL0910,122,Adelie Penguin (Pygoscelis adeliae),Anvers,Torgersen,"Adult, 1 Egg Stage",N66A2,No,2009-11-17,37.7,19.8,198.0,3500.0,MALE,9.11066,-26.42563,Nest never observed with full clutch.
130,PAL0910,131,Adelie Penguin (Pygoscelis adeliae),Anvers,Torgersen,"Adult, 1 Egg Stage",N73A1,No,2009-11-23,38.5,17.9,190.0,3325.0,FEMALE,8.98460,-25.57956,Nest never observed with full clutch.


#### Show some info of the new created dataframe with df.info()

In [8]:
df.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 34 entries, 6 to 341
Data columns (total 17 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   studyName            34 non-null     object 
 1   Sample Number        34 non-null     int64  
 2   Species              34 non-null     object 
 3   Region               34 non-null     object 
 4   Island               34 non-null     object 
 5   Stage                34 non-null     object 
 6   Individual ID        34 non-null     object 
 7   Clutch Completion    34 non-null     object 
 8   Date Egg             34 non-null     object 
 9   Culmen Length (mm)   34 non-null     float64
 10  Culmen Depth (mm)    34 non-null     float64
 11  Flipper Length (mm)  34 non-null     float64
 12  Body Mass (g)        34 non-null     float64
 13  Sex                  34 non-null     object 
 14  Delta 15 N (o/oo)    34 non-null     float64
 15  Delta 13 C (o/oo)    34 non-null     floa

##### We see that we have a problem. At first the data consisted of 344 rows, but the new dataframe consist of 34 rows (as we can see from df.info() above). This means we might have deleted too many rows. Whats going wrong here?

##### Look at the original Comments-column by scrolling upwards to your previously created df.info() - output. We can see that the original Comments-column only had 54 rows, compared to the other columns, which had like ~300 rows. So if we are deleting rows of the whole dataframe based on the NaNs of the Comments-column we lose a lot of data.

##### So lets see how we can fix this problem. Lets first check whats in the Comments column:

In [9]:
print(df["Comments"])


6      Nest never observed with full clutch.
7      Nest never observed with full clutch.
28     Nest never observed with full clutch.
29     Nest never observed with full clutch.
38     Nest never observed with full clutch.
68     Nest never observed with full clutch.
69     Nest never observed with full clutch.
120    Nest never observed with full clutch.
121    Nest never observed with full clutch.
130    Nest never observed with full clutch.
131    Nest never observed with full clutch.
138    Nest never observed with full clutch.
139    Nest never observed with full clutch.
162    Nest never observed with full clutch.
163    Nest never observed with full clutch.
192    Nest never observed with full clutch.
193    Nest never observed with full clutch.
198    Nest never observed with full clutch.
199    Nest never observed with full clutch.
270    Nest never observed with full clutch.
276    Nest never observed with full clutch.
277    Nest never observed with full clutch.
292    Nes

#### Okay now we have a feel whats in the Comments-column. In the next steps we are gonna load again the whole file and fill in the Nan values of the Comments-column, as this column might be the less significant column.

#### Lets start over. We are gonna load the whole penguins file again by reading in the parquet file.

In [10]:
df = pd.read_parquet(filename, engine="pyarrow")
df.describe()


,Sample Number,Culmen Length (mm),Culmen Depth (mm),Flipper Length (mm),Body Mass (g),Delta 15 N (o/oo),Delta 13 C (o/oo)
count,344.000000,342.000000,342.000000,342.000000,342.000000,330.000000,331.000000
mean,63.151163,43.921930,17.151170,200.915205,4201.754386,8.733382,-25.686292
std,40.430199,5.459584,1.974793,14.061714,801.954536,0.551770,0.793961
min,1.000000,32.100000,13.100000,172.000000,2700.000000,7.632200,-27.018540
25%,29.000000,39.225000,15.600000,190.000000,3550.000000,8.299890,-26.320305
50%,58.000000,44.450000,17.300000,197.000000,4050.000000,8.652405,-25.833520
75%,95.250000,48.500000,18.700000,213.000000,4750.000000,9.172123,-25.062050
max,152.000000,59.600000,21.500000,231.000000,6300.000000,10.025440,-23.787670


#### Perfect, we have the whole dataframe loaded again. Lets fill in the NaN values of the Comments-column with e.g. "No comment found" :

In [11]:
# We replace all the NaN values within the Comments-column with "No comment found"
df["Comments"] = df["Comments"].fillna("No comment found")


#### This is one of the many possible solutions to tackle this problem. Just an example of how to deal with missing values.

In [12]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 344 entries, 0 to 343
Data columns (total 17 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   studyName            344 non-null    object 
 1   Sample Number        344 non-null    int64  
 2   Species              344 non-null    object 
 3   Region               344 non-null    object 
 4   Island               344 non-null    object 
 5   Stage                344 non-null    object 
 6   Individual ID        344 non-null    object 
 7   Clutch Completion    344 non-null    object 
 8   Date Egg             344 non-null    object 
 9   Culmen Length (mm)   342 non-null    float64
 10  Culmen Depth (mm)    342 non-null    float64
 11  Flipper Length (mm)  342 non-null    float64
 12  Body Mass (g)        342 non-null    float64
 13  Sex                  333 non-null    object 
 14  Delta 15 N (o/oo)    330 non-null    float64
 15  Delta 13 C (o/oo)    331 non-null    flo

#### Now we check again which rows are Nan, and delete the corresponding rows from the whole table:

In [13]:
select = list(df.isna().sum() > 0)
df = df.dropna(subset=df.columns[select], axis="rows")
df

,studyName,Sample Number,Species,Region,Island,Stage,Individual ID,Clutch Completion,Date Egg,Culmen Length (mm),Culmen Depth (mm),Flipper Length (mm),Body Mass (g),Sex,Delta 15 N (o/oo),Delta 13 C (o/oo),Comments
1,PAL0708,2,Adelie Penguin (Pygoscelis adeliae),Anvers,Torgersen,"Adult, 1 Egg Stage",N1A2,Yes,2007-11-11,39.5,17.4,186.0,3800.0,FEMALE,8.94956,-24.69454,No comment found
2,PAL0708,3,Adelie Penguin (Pygoscelis adeliae),Anvers,Torgersen,"Adult, 1 Egg Stage",N2A1,Yes,2007-11-16,40.3,18.0,195.0,3250.0,FEMALE,8.36821,-25.33302,No comment found
4,PAL0708,5,Adelie Penguin (Pygoscelis adeliae),Anvers,Torgersen,"Adult, 1 Egg Stage",N3A1,Yes,2007-11-16,36.7,19.3,193.0,3450.0,FEMALE,8.76651,-25.32426,No comment found
5,PAL0708,6,Adelie Penguin (Pygoscelis adeliae),Anvers,Torgersen,"Adult, 1 Egg Stage",N3A2,Yes,2007-11-16,39.3,20.6,190.0,3650.0,MALE,8.66496,-25.29805,No comment found
6,PAL0708,7,Adelie Penguin (Pygoscelis adeliae),Anvers,Torgersen,"Adult, 1 Egg Stage",N4A1,No,2007-11-15,38.9,17.8,181.0,3625.0,FEMALE,9.18718,-25.21799,Nest never observed with full clutch.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
339,PAL0910,64,Chinstrap penguin (Pygoscelis antarctica),Anvers,Dream,"Adult, 1 Egg Stage",N98A2,Yes,2009-11-19,55.8,19.8,207.0,4000.0,MALE,9.70465,-24.53494,No comment found
340,PAL0910,65,Chinstrap penguin (Pygoscelis antarctica),Anvers,Dream,"Adult, 1 Egg Stage",N99A1,No,2009-11-21,43.5,18.1,202.0,3400.0,FEMALE,9.37608,-24.40753,Nest never observed with full clutch.
341,PAL0910,66,Chinstrap penguin (Pygoscelis antarctica),Anvers,Dream,"Adult, 1 Egg Stage",N99A2,No,2009-11-21,49.6,18.2,193.0,3775.0,MALE,9.46180,-24.70615,Nest never observed with full clutch.
342,PAL0910,67,Chinstrap penguin (Pygoscelis antarctica),Anvers,Dream,"Adult, 1 Egg Stage",N100A1,Yes,2009-11-21,50.8,19.0,210.0,4100.0,MALE,9.98044,-24.68741,No comment found


In [14]:
df.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 324 entries, 1 to 343
Data columns (total 17 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   studyName            324 non-null    object 
 1   Sample Number        324 non-null    int64  
 2   Species              324 non-null    object 
 3   Region               324 non-null    object 
 4   Island               324 non-null    object 
 5   Stage                324 non-null    object 
 6   Individual ID        324 non-null    object 
 7   Clutch Completion    324 non-null    object 
 8   Date Egg             324 non-null    object 
 9   Culmen Length (mm)   324 non-null    float64
 10  Culmen Depth (mm)    324 non-null    float64
 11  Flipper Length (mm)  324 non-null    float64
 12  Body Mass (g)        324 non-null    float64
 13  Sex                  324 non-null    object 
 14  Delta 15 N (o/oo)    324 non-null    float64
 15  Delta 13 C (o/oo)    324 non-null    flo

#### This time we didnt lose as much data as before!

#### For the next question we use a regex expression to shorten the name of the penguins. (one may use chatgpt for finding regex expressions!)

In [15]:
import re

regex = re.compile("^([^(]+)")
out = re.search(regex, "Chinstrap penguin (Pygoscelis antarctica)")
out.group()


'Chinstrap penguin '

In [16]:
def extract(regex, msg):
    out = re.search(regex, msg)
    return out.group()


#### We apply the regex expression onto the Species-column, as the names of the penguins are in this column:

In [17]:
df["Species"] = df["Species"].apply(lambda x: extract(regex=regex, msg=x))


#### Finally we save the cleaned file with a timestamp into our output directory!

In [18]:
from datetime import datetime

tag = datetime.now().strftime("%Y%m%d-%H%M") + ".csv"
output = outputdir / tag
df.to_csv(output, index=False)
